## Tratamento da regionalização da execução orçamentária

Nesse notebook são tratados os dados da regionalização da execução orçamentária da Prefeitura Municipal de São Paulo para o ano de 2024

In [1]:
import pandas as pd
import numpy as np
from utils.load_csv import load_csv
from utils.load_shapefile import load_shapefile
from utils.save_csv import save_csv

In [2]:
df_reg_orcamento = load_csv("reg_orcamento_2024.csv")

c:\Users\x050258\projetos\dash_orcamento_melhores_praticas_estagio\utils\load_csv.py:18: DtypeWarning: Columns (27,28,29,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path, **read_kwargs)


#### Filtro regionalizável

Filtra a base de dados para conter apenas despesas regionalizáveis

In [3]:
df_reg_orcamento["TIPO_REGIONALIZAÇÃO"].unique()

array(['Despesa Regionalizável', nan, 'Despesa Não-Regionalizável'],
      dtype=object)

In [4]:
df_orcamento_regionalizavel = df_reg_orcamento[df_reg_orcamento["TIPO_REGIONALIZAÇÃO"] == "Despesa Regionalizável"].copy()

df_orcamento_regionalizavel["TIPO_REGIONALIZAÇÃO"].unique()

array(['Despesa Regionalizável'], dtype=object)

In [5]:
df_orcamento_regionalizavel

,COD_EMPRESA_PMSP,COD_EMPENHO,ANO_EMPENHO,CÓDIGO_NLP,ANO_LIQUIDAÇÃO,DATA_LIQUIDAÇÃO,CÓDIGO_ÓRGÃO,SIGLA_ÓRGÃO,DESCRIÇÃO_ÓRGÃO,CÓDIGO_UNIDADE,...,CÓDIGO_EXERCÍCIO_FONTE,CÓDIGO_DESTINAÇÃO_RECURSO,CÓDIGO_VÍNCULO_PMSP,CÓDIGO_TIPO_CRÉDITO_ORÇAMENTÁRIO,REGIÃO,SUBPREFEITURA,DISTRITO,TIPO_REGIONALIZAÇÃO,VALOR_DETALHAMENTO_AÇÃO,DATA_EXTRAÇÃO
0,4,197,2024,362,2024,2024-01-30 00:00:00.0000000,2,HSPM,Hospital do Servidor Público Municipal,10,...,1,500,9001,0,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,"1792,03",03/07/2025
1,4,197,2024,893,2024,2024-02-28 00:00:00.0000000,2,HSPM,Hospital do Servidor Público Municipal,10,...,1,500,9001,0,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,"1780,47",03/07/2025
2,4,197,2024,1973,2024,2024-04-05 00:00:00.0000000,2,HSPM,Hospital do Servidor Público Municipal,10,...,1,500,9001,0,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,"1667,19",03/07/2025
3,4,197,2024,2430,2024,2024-04-29 00:00:00.0000000,2,HSPM,Hospital do Servidor Público Municipal,10,...,1,500,9001,0,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,"1667,19",03/07/2025
4,4,2028,2024,3142,2024,2024-05-29 00:00:00.0000000,2,HSPM,Hospital do Servidor Público Municipal,10,...,1,500,9001,0,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,"1782,02",03/07/2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
530898,1,30563,2024,348205,2024,2024-12-16 00:00:00.0000000,98,FUNDURB,Fundo de Desenvolvimento Urbano,29,...,1,759,402,0,Norte,Subprefeitura Freguesia/Brasilândia,Supra-Distrital,Despesa Regionalizável,"6364,98",03/07/2025
530946,1,28251,2024,57235,2024,2024-02-28 00:00:00.0000000,99,FUNDIP,Fundo Municipal de Iluminação Pública,10,...,1,751,253,0,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,"4412778,26",03/07/2025
530947,1,28251,2024,57235,2024,2024-02-28 00:00:00.0000000,99,FUNDIP,Fundo Municipal de Iluminação Pública,10,...,1,751,253,0,Leste,Subprefeitura São Mateus,Supra-Distrital,Despesa Regionalizável,"333997,94",03/07/2025
530948,1,28251,2024,57235,2024,2024-02-28 00:00:00.0000000,99,FUNDIP,Fundo Municipal de Iluminação Pública,10,...,1,751,253,0,Leste,Subprefeitura São Miguel Paulista,Supra-Distrital,Despesa Regionalizável,"1709010,06",03/07/2025


#### Transformação da variável de gasto

Transforma a variável de gasto da base de dados de string para float

In [6]:
df_orcamento_regionalizavel["VALOR_DETALHAMENTO_AÇÃO"].unique()

array(['1792,03', '1780,47', '1667,19', ..., '333997,94', '1709010,06',
       '444134,83'], shape=(148398,), dtype=object)

In [7]:
df_orcamento_regionalizavel["VALOR_DETALHAMENTO_AÇÃO"] = df_orcamento_regionalizavel["VALOR_DETALHAMENTO_AÇÃO"].str.replace(".", "").str.replace(",", ".").astype(float)

pd.options.display.float_format = '{:,.2f}'.format

df_orcamento_regionalizavel["VALOR_DETALHAMENTO_AÇÃO"].describe()

count       354,299.00
mean        128,592.39
std       2,417,553.63
min               0.00
25%             657.60
50%           6,000.00
75%          49,453.42
max     768,185,644.10
Name: VALOR_DETALHAMENTO_AÇÃO, dtype: float64

In [8]:
np.set_printoptions(suppress = True)

df_orcamento_regionalizavel["VALOR_DETALHAMENTO_AÇÃO"].unique()

array([   1792.03,    1780.47,    1667.19, ...,  333997.94, 1709010.06,
        444134.83], shape=(148398,))

#### Normalização das subprefeituras

Trata os nomes das subprefreituras na base de dados da regionalização da execução orçamentária para estar igual aos nomes no shapefile das subprefeituras

In [9]:
df_orcamento_regionalizavel["SUBPREFEITURA"].unique()

array(['Subprefeitura Sé', 'Subprefeitura Vila Mariana',
       'Subprefeitura Santana/Tucuruvi', 'Subprefeitura Campo Limpo',
       'Subprefeitura Itaim Paulista', 'Subprefeitura Pirituba/Jaraguá',
       'Subprefeitura Sapopemba', 'Subprefeitura Capela do Socorro',
       'Subprefeitura Lapa', 'Subprefeitura Jaçanã/Tremembé',
       'Subprefeitura Penha', 'Subprefeitura São Miguel Paulista',
       'Subprefeitura Perus/Anhanguera', 'Subprefeitura Butantã',
       'Subprefeitura de Guaianases', 'Subprefeitura Cidade Ademar',
       'Subprefeitura São Mateus', 'Subprefeitura Santo Amaro',
       'Subprefeitura Aricanduva/Formosa/Carrão',
       'Subprefeitura Casa Verde/Cachoeirinha', 'Subprefeitura Ipiranga',
       'Subprefeitura Itaquera', 'Subprefeitura Jabaquara',
       'Subprefeitura de Vila Prudente',
       'Subprefeitura Cidade Tiradentes',
       'Subprefeitura Ermelino Matarazzo',
       'Subprefeitura Freguesia/Brasilândia', 'Subprefeitura Parelheiros',
       "Subprefeit

In [10]:
supra_subprefs = [
    "Supra Subprefeitura Centro",
    "Supra Subprefeitura Leste",
    "Supra Subprefeitura Norte",
    "Supra Subprefeitura Oeste",
    "Supra Subprefeitura Sul"]

df_orcamento_regionalizavel = df_orcamento_regionalizavel[~df_orcamento_regionalizavel["SUBPREFEITURA"].isin(supra_subprefs)]

df_orcamento_regionalizavel["SUBPREFEITURA"].unique()

array(['Subprefeitura Sé', 'Subprefeitura Vila Mariana',
       'Subprefeitura Santana/Tucuruvi', 'Subprefeitura Campo Limpo',
       'Subprefeitura Itaim Paulista', 'Subprefeitura Pirituba/Jaraguá',
       'Subprefeitura Sapopemba', 'Subprefeitura Capela do Socorro',
       'Subprefeitura Lapa', 'Subprefeitura Jaçanã/Tremembé',
       'Subprefeitura Penha', 'Subprefeitura São Miguel Paulista',
       'Subprefeitura Perus/Anhanguera', 'Subprefeitura Butantã',
       'Subprefeitura de Guaianases', 'Subprefeitura Cidade Ademar',
       'Subprefeitura São Mateus', 'Subprefeitura Santo Amaro',
       'Subprefeitura Aricanduva/Formosa/Carrão',
       'Subprefeitura Casa Verde/Cachoeirinha', 'Subprefeitura Ipiranga',
       'Subprefeitura Itaquera', 'Subprefeitura Jabaquara',
       'Subprefeitura de Vila Prudente',
       'Subprefeitura Cidade Tiradentes',
       'Subprefeitura Ermelino Matarazzo',
       'Subprefeitura Freguesia/Brasilândia', 'Subprefeitura Parelheiros',
       "Subprefeit

In [11]:
gdf = load_shapefile("subprefeituras_geosampa.shp")

In [12]:
gdf.head()

,sp_areamt,sp_areakmt,sp_codigo,sp_id,sp_sigla,sp_nome,geometry
0,"31,980,202.74",32.00,03,2.00,FO,FREGUESIA-BRASILANDIA,"POLYGON ((327340.628 7399133.313, 327331.514 7..."
1,"27,232,342.53",27.00,04,3.00,CV,CASA VERDE-CACHOEIRINHA,"POLYGON ((329084.795 7402363.669, 329086.123 7..."
2,"40,638,699.56",41.00,08,13.00,LA,LAPA,"POLYGON ((321633.729 7393535.365, 321633.031 7..."
3,"56,889,310.75",57.00,01,31.00,PR,PERUS,"POLYGON ((325464.932 7409329.344, 325464.513 7..."
4,"26,665,465.09",27.00,09,14.00,SE,SE,"POLYGON ((330197.017 7396087.885, 330211.849 7..."


In [13]:
gdf["sp_nome"].unique()

array(['FREGUESIA-BRASILANDIA', 'CASA VERDE-CACHOEIRINHA', 'LAPA',
       'PERUS', 'SE', 'SANTANA-TUCURUVI', 'PINHEIROS', 'JACANA-TREMEMBE',
       'VILA MARIA-VILA GUILHERME', 'MOOCA', 'IPIRANGA', 'VILA MARIANA',
       'PENHA', 'ERMELINO MATARAZZO', 'SAO MIGUEL', 'ITAQUERA',
       'ARICANDUVA-FORMOSA-CARRAO', 'SAO MATEUS', 'VILA PRUDENTE',
       'SAPOPEMBA', 'PARELHEIROS', 'CAMPO LIMPO', 'CAPELA DO SOCORRO',
       'CIDADE ADEMAR', 'JABAQUARA', 'SANTO AMARO', 'ITAIM PAULISTA',
       'GUAIANASES', 'CIDADE TIRADENTES', 'BUTANTA', 'PIRITUBA-JARAGUA',
       'M BOI MIRIM'], dtype=object)

In [14]:
mapper = {"Subprefeitura Sé": "SE",
          "Subprefeitura Vila Mariana": "VILA MARIANA",
          "Subprefeitura Santana/Tucuruvi": "SANTANA-TUCURUVI",
          "Subprefeitura Campo Limpo": "CAMPO LIMPO",
          "Subprefeitura Itaim Paulista": "ITAIM PAULISTA",
          "Subprefeitura Pirituba/Jaraguá": "PIRITUBA-JARAGUA",
          "Subprefeitura Sapopemba": "SAPOPEMBA",
          "Subprefeitura Capela do Socorro": "CAPELA DO SOCORRO",
          "Subprefeitura Lapa": "LAPA",
          "Subprefeitura Jaçanã/Tremembé": "JACANA-TREMEMBE",
          "Subprefeitura Penha": "PENHA",
          "Subprefeitura São Miguel Paulista": "SAO MIGUEL",
          "Subprefeitura Perus/Anhanguera": "PERUS",
          "Subprefeitura Butantã": "BUTANTA",
          "Subprefeitura de Guaianases": "GUAIANASES",
          "Subprefeitura São Mateus": "SAO MATEUS",
          "Subprefeitura Cidade Ademar": "CIDADE ADEMAR",
          "Subprefeitura de Vila Prudente": "VILA PRUDENTE",
          "Subprefeitura Ipiranga": "IPIRANGA",
          "Subprefeitura Pinheiros": "PINHEIROS",
          "Subprefeitura Casa Verde/Cachoeirinha": "CASA VERDE-CACHOEIRINHA",
          "Subprefeitura Freguesia/Brasilândia": "FREGUESIA-BRASILANDIA",
          "Subprefeitura M'Boi Mirim": "M BOI MIRIM",
          "Subprefeitura Vila Maria/Vila Guilherme": "VILA MARIA-VILA GUILHERME",
          "Subprefeitura Aricanduva/Formosa/Carrão": "ARICANDUVA-FORMOSA-CARRAO",
          "Subprefeitura Santo Amaro": "SANTO AMARO",
          "Subprefeitura Mooca": "MOOCA",
          "Subprefeitura Ermelino Matarazzo": "ERMELINO MATARAZZO",
          "Subprefeitura Cidade Tiradentes": "CIDADE TIRADENTES",
          "Subprefeitura Itaquera": "ITAQUERA",
          "Subprefeitura Jabaquara": "JABAQUARA",
          "Subprefeitura Parelheiros": "PARELHEIROS"}

df_orcamento_regionalizavel.loc[:, "SUBPREFEITURA"] = df_orcamento_regionalizavel["SUBPREFEITURA"].map(mapper)

df_orcamento_regionalizavel["SUBPREFEITURA"].unique()

array(['SE', 'VILA MARIANA', 'SANTANA-TUCURUVI', 'CAMPO LIMPO',
       'ITAIM PAULISTA', 'PIRITUBA-JARAGUA', 'SAPOPEMBA',
       'CAPELA DO SOCORRO', 'LAPA', 'JACANA-TREMEMBE', 'PENHA',
       'SAO MIGUEL', 'PERUS', 'BUTANTA', 'GUAIANASES', 'CIDADE ADEMAR',
       'SAO MATEUS', 'SANTO AMARO', 'ARICANDUVA-FORMOSA-CARRAO',
       'CASA VERDE-CACHOEIRINHA', 'IPIRANGA', 'ITAQUERA', 'JABAQUARA',
       'VILA PRUDENTE', 'CIDADE TIRADENTES', 'ERMELINO MATARAZZO',
       'FREGUESIA-BRASILANDIA', 'PARELHEIROS', 'M BOI MIRIM', 'MOOCA',
       'PINHEIROS', 'VILA MARIA-VILA GUILHERME'], dtype=object)

In [15]:
save_csv(df_orcamento_regionalizavel, "orcamento_tratado_2024.csv")

Data saved to data\orcamento_tratado_2024.csv
